<a href="https://colab.research.google.com/github/mnocerino23/Wildfire-Forecaster/blob/main/Final_Prepping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this Jupyter Notebook, I will start by filling in missing county data into the first larger wildfire dataset through the use of a function that returns the California county given the coordinates (latitude and longitude). I then add the feature Receives_Snow to the dataset which denotes whether or not the county experiences snowfall. 

For those county's that do experience snowfall, I find the closest snow station to the fire location and add on snowpack related features for the year of the fire that I extracted from CDEC earlier in this project.


After this, I  decide how to handle null values and make some final changes to the datset before we begin applying machine learning techniques.

In [162]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

#Read in the larger dataset (2001-2015) including NOAA weather data and engineered features
wildfires1 = pd.read_csv('/content/drive/MyDrive/Data_Science_Projects/Wildfires/CA_wildfires_with_weather.csv')

#Read in the smaller dataset (2016-2019) including NOAA weather data and engineered features
wildfires2 = pd.read_csv('/content/drive/MyDrive/Data_Science_Projects/Wildfires/CA_wildfires_recent_with_weather.csv')

#Read in the snow data that I gathered in the CDEC snow data jupyter notebook
snow = pd.read_csv('/content/drive/MyDrive/Data_Science_Projects/Wildfires/CA_SnowData.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The dictionary below contains all 58 counties in the state of California mapped to a 0 (meaning they do not receive snow) and a 1 (if the county receives snowfall). 

To add in snowfall data, I will use the following procedure:


In [163]:
receives_snow = {'Alameda': 0, 'Alpine': 1, 'Amador':1, 'Butte': 0,
                 'Calaveras': 1, 'Colusa': 0, 'Contra Costa': 0,
                 'Del Norte': 0, 'El Dorado': 1,
                 'Fresno': 1 , 'Glenn': 0,'Humboldt': 0,'Imperial':0,
                 'Inyo': 1, 'Kern': 1, 'Kings': 0, 'Lake': 0,'Lassen': 1 ,'Los Angeles': 0,
                 'Madera': 1, 'Marin': 0, 'Mariposa': 1, 'Mendocino': 0,
                 'Merced': 0, 'Modoc': 1,'Mono': 1, 'Monterey': 0,
                 'Napa': 0, 'Nevada': 1, 'Orange': 0, 'Placer': 1, 'Plumas': 1,
                 'Riverside': 0, 'Sacramento': 0, 'San Benito': 0, 
                 'San Bernardino': 0, 'San Diego': 0, 'San Francisco': 0, 
                 'San Joaquin': 0, 'San Luis Obispo': 0, 'San Mateo': 0,
                 'Santa Barbara': 0, 'Santa Clara': 0, 'Santa Cruz': 0,
                 'Shasta': 1, 'Sierra': 1 , 'Siskiyou': 1, 'Solano': 0, 'Sonoma': 0, 
                 'Stanislaus': 1, 'Sutter': 0, 'Tehama': 1, 'Trinity': 1, 
                 'Tulare':1, 'Tuolumne': 1, 'Ventura': 0, 'Yolo': 0, 'Yuba': 1}

In [164]:
#The smaller more recent dataset has county name for every fire. However, the larger dataset with fires from 
#2001-2015 has many NaN for county's. Before proceeding with the process, I try to add in missing counties by 
#using the geopy library and creating a function (with help from stack overflow) to get a fire's county within
#california given it's longitude and latitude. Below I show an example to confirm that the function works.

In [165]:
wildfires1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61572 entries, 0 to 61571
Data columns (total 32 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  61572 non-null  int64  
 1   Year                        61572 non-null  int64  
 2   Name                        57233 non-null  object 
 3   AcresBurned                 61572 non-null  float64
 4   Fire Size Rank              61572 non-null  object 
 5   Cause                       61572 non-null  object 
 6   SOURCE_REPORTING_UNIT_NAME  61572 non-null  object 
 7   DaysBurn                    61572 non-null  float64
 8   Discovery Month             61572 non-null  object 
 9   Discovered DOY              61572 non-null  float64
 10  Contained Month             61572 non-null  object 
 11  Contained DOY               61572 non-null  float64
 12  Latitude                    61572 non-null  float64
 13  Longitude                   615

In [166]:
import geopy as gp
from geopy import Nominatim

def coordinates_to_county(lat, long, geolocater):
  location = geolocater.reverse((lat, long))
  #print(location)
  if 'county' in location.raw['address'].keys():
    county = location.raw['address']['county']

    #split the string into its component words so we can get rid of the 'County' at the end of each county name easily
    county = county.split()[0:-1]
    corrected = ''
    count = 0
    for item in county: 
      if count == 0:
        corrected += item
        count += 1
      else:
        corrected += ' ' + item
    return corrected
  else:
    return float('nan')

geolocater = gp.Nominatim(user_agent='my-application')
coordinates_to_county(38.93305556, -120.40444444, geolocater)

'El Dorado'

In [237]:
for index, row in wildfires1.iterrows():
  #If the value is null (has type float), we use the created function to get its county
  if type(wildfires1.at[index,'County']) == float:
    wildfires1.at[index,'County'] = coordinates_to_county(wildfires1.at[index,'Latitude'], wildfires1.at[index,'Longitude'], geolocater)

In [246]:
#Notice how after running the cell above, we only have 44 null values for county.
print(wildfires1.isnull().sum())

Unnamed: 0                        0
Year                              0
Name                           4339
AcresBurned                       0
Fire Size Rank                    0
Cause                             0
SOURCE_REPORTING_UNIT_NAME        0
DaysBurn                          0
Discovery Month                   0
Discovered DOY                    0
Contained Month                   0
Contained DOY                     0
Latitude                          0
Longitude                         0
County                           44
CountyIds                     23073
State                             0
OWNER_DESCR                       0
NOAA Station                      0
Link                              0
AWND                           3107
CLDD                           2023
DP10                           1936
DX90                           2023
PRCP                           1936
TAVG                           2023
TMAX                           2023
TMIN                        

If the county of the fire is known, we add in the value for that county key in the receives_snow dictionary. 1 means the county receives snow, and a 0 denotes that the county does not get snow.

In [247]:
wildfires1['Receives Snow'] = wildfires1['County'].apply(lambda x: receives_snow[x] if x in list(receives_snow.keys()) else float('nan'))
wildfires2['Receives Snow'] = wildfires2['County'].apply(lambda x: receives_snow[x] if x in list(receives_snow.keys()) else float('nan'))

We will be adding on the data that I obtained from California department of water resources which I packed into an organized .csv file in the California_CDEC_SnowData.ipynb file within this repository. Once again the features added are as follows:


1.   March Snowpack (Mar_SP)
2.   March Snowpack Water Content (Mar_WC)
3.   March Snowpack Density (Mar_Dens)

Note: We are using snowpack data from March because that is generally when snowpack within the mountainous areas of California is at its peak.



In [249]:
wildfires1['Snow Station'] = ''
wildfires1['River Basin'] = ''
wildfires1['Mar_SP'] = ''
wildfires1['Mar_WC'] = ''
wildfires1['Mar_Dens'] = ''

wildfires2['Snow Station'] = ''
wildfires2['River Basin'] = ''
wildfires2['Mar_SP'] = ''
wildfires2['Mar_WC'] = ''
wildfires2['Mar_Dens'] = ''

In the code below, we find the closest snow station to each fire using the haversine distance (distance between coordinates on earth) and add on our snow related features for the year of the fire. We apply this code to both datasets to produce our final dataset which now has temperature, snow, and engineered features.

In [250]:
!pip install haversine
import haversine as hs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [297]:
for index, row in wildfires1.iterrows():
  #Use the first if statement to weed out counties that do not receive snow. They will get zeroes for snow related stats
  if wildfires1.at[index,'Receives Snow'] == 1:
    #initialize to a super large number so that condition of being less than this will be met right away
    min_dist = 1000000000
    w = []
    w.append(float(wildfires1.at[index, 'Latitude']))
    w.append(float(wildfires1.at[index, 'Longitude']))
    wildfire_coordinates = tuple(w)

    for index2, row in snow.iterrows():
      cdec_station = snow.at[index2, 'CDEC Station']
      river_basin = snow.at[index2,'River Basin']
      year = str(wildfires1.at[index, 'Year'])
      data = snow.at[index2, year]
      s = []
      s.append(float(snow.at[index2, 'Latitude']))
      s.append(float(snow.at[index2, 'Longitude']))
      station_coordinates = tuple(s)

      dist = hs.haversine(station_coordinates, wildfire_coordinates)
      #Find the CDEC snow station with the minimum distance from the fire to add in
      #snow data from the year
      if dist < min_dist:
        min_dist = dist
        min_station = cdec_station
        basin = river_basin
        
        if type(data) != float:
          snow_data = data[1:-1]
          snow_data = tuple(map(float, snow_data.split(', ')))

    wildfires1.at[index, 'Snow Station'] = min_station
    wildfires1.at[index, 'River Basin'] = basin
    wildfires1.at[index, 'Mar_SP'] = snow_data[0]
    wildfires1.at[index, 'Mar_WC'] = snow_data[1]
    wildfires1.at[index, 'Mar_Dens'] = snow_data[2]

  elif wildfires1.at[index,'Receives Snow'] == 0:
      wildfires1.at[index, 'Mar_SP'] = 0
      wildfires1.at[index, 'Mar_WC'] = 0
      wildfires1.at[index, 'Mar_Dens'] = 0
      wildfires1.at[index, 'Snow Station'] = ''
      wildfires1.at[index, 'River Basin'] = ''

In [298]:
for index, row in wildfires2.iterrows():
  #Use the first if statement to weed out counties that do not receive snow. They will get zeroes for snow related stats
  if wildfires2.at[index,'Receives Snow'] == 1:
    #initialize to a super large number so that condition of being less than this will be met right away
    min_dist = 1000000000
    w = []
    w.append(float(wildfires2.at[index, 'Latitude']))
    w.append(float(wildfires2.at[index, 'Longitude']))
    wildfire_coordinates = tuple(w)

    for index2, row in snow.iterrows():
      cdec_station = snow.at[index2, 'CDEC Station']
      river_basin = snow.at[index2,'River Basin']
      year = str(wildfires2.at[index, 'Year'])
      data = snow.at[index2, year]
      s = []
      s.append(float(snow.at[index2, 'Latitude']))
      s.append(float(snow.at[index2, 'Longitude']))
      station_coordinates = tuple(s)

      dist = hs.haversine(station_coordinates, wildfire_coordinates)
      #Find the CDEC snow station with the minimum distance from the fire to add in
      #snow data from the year
      if dist < min_dist:
        min_dist = dist
        min_station = cdec_station
        basin = river_basin
        
        if type(data) != float:
          snow_data = data[1:-1]
          snow_data = tuple(map(float, snow_data.split(', ')))

    wildfires2.at[index, 'Snow Station'] = min_station
    wildfires2.at[index, 'River Basin'] = basin
    wildfires2.at[index, 'Mar_SP'] = snow_data[0]
    wildfires2.at[index, 'Mar_WC'] = snow_data[1]
    wildfires2.at[index, 'Mar_Dens'] = snow_data[2]

  elif wildfires2.at[index,'Receives Snow'] == 0:
      wildfires2.at[index, 'Mar_SP'] = 0
      wildfires2.at[index, 'Mar_WC'] = 0
      wildfires2.at[index, 'Mar_Dens'] = 0
      wildfires2.at[index, 'Snow Station'] = ''
      wildfires2.at[index, 'River Basin'] = ''

In [302]:
wildfires1.head(10)

,Unnamed: 0,Year,Name,AcresBurned,Fire Size Rank,Cause,SOURCE_REPORTING_UNIT_NAME,DaysBurn,Discovery Month,Discovered DOY,...,PRCP_6M,PRCP_RS,DX90_2M,DP10_2M,Receives Snow,Snow Station,River Basin,Mar_SP,Mar_WC,Mar_Dens
0,0,2005,FOUNTAIN,0.10,A,Miscellaneous,Plumas National Forest,1.0,Feb,33.0,...,27.89,49.06,0.0,19.0,1.0,eureka_lake,Feather,79.6,34.0,0.43
1,1,2004,PIGEON,0.25,A,Lightning,Eldorado National Forest,1.0,May,133.0,...,14.37,14.76,0.0,3.0,1.0,ward_creek_2,Lake Tahoe,108.6,38.1,0.35
2,2,2004,SLACK,0.10,A,Debris Burning,Eldorado National Forest,1.0,Jun,152.0,...,36.71,40.37,0.0,11.0,1.0,ward_creek_2,Lake Tahoe,108.6,38.1,0.35
3,3,2004,DEER,0.10,A,Lightning,Eldorado National Forest,5.0,Jun,180.0,...,13.63,14.76,0.0,3.0,1.0,echo_summit,American,87.2,28.4,0.33
4,4,2004,STEVENOT,0.10,A,Lightning,Eldorado National Forest,5.0,Jun,180.0,...,13.63,14.76,0.0,3.0,1.0,echo_summit,American,87.2,28.4,0.33
5,5,2004,HIDDEN,0.10,A,Lightning,Eldorado National Forest,1.0,Jul,182.0,...,8.31,14.76,0.0,3.0,1.0,echo_summit,American,87.2,28.4,0.33
6,6,2004,FORK,0.10,A,Lightning,Eldorado National Forest,1.0,Jul,183.0,...,8.31,14.76,0.0,3.0,1.0,echo_summit,American,87.2,28.4,0.33
7,7,2005,SLATE,0.80,B,Debris Burning,Shasta-Trinity National Forest,1.0,Mar,67.0,...,25.91,32.72,0.0,12.0,1.0,red_rock_mountain,Trinity,110.0,49.0,0.45
8,8,2005,SHASTA,1.00,B,Debris Burning,Shasta-Trinity National Forest,1.0,Mar,74.0,...,25.91,32.72,0.0,12.0,1.0,mt_shasta,Sacramento,61.1,54.0,0.88
9,9,2004,TANGLEFOOT,0.10,A,Lightning,Eldorado National Forest,1.0,Jul,183.0,...,8.31,14.76,0.0,3.0,1.0,hams_station,Mokelumne,8.0,3.0,0.38


In [301]:
wildfires2.head(10)

,Unnamed: 0,Year,Name,AcresBurned,Fire Size Rank,Cause,SOURCE_REPORTING_UNIT_NAME,DaysBurn,Discovery Month,Discovered DOY,...,PRCP_6M,PRCP_RS,DX90_2M,DP10_2M,Receives Snow,Snow Station,River Basin,Mar_SP,Mar_WC,Mar_Dens
0,0,2016,Soberanes Fire,132127.0,G,NaN,NaN,83.0,Jul,NaN,...,14.11,21.42,0.0,1.0,0,,,0,0,0
1,1,2016,Erskine Fire,48019.0,G,NaN,NaN,18.0,Jun,NaN,...,4.68,4.88,15.0,4.0,1,mineral_king,Kaweah,36.0,16.0,0.44
2,2,2016,Chimney Fire,46344.0,G,NaN,NaN,24.0,Aug,NaN,...,2.52,8.09,43.0,0.0,0,,,0,0,0
3,3,2016,Blue Cut Fire,36274.0,G,NaN,NaN,7.0,Aug,NaN,...,3.41,6.45,43.0,0.0,0,,,0,0,0
4,4,2016,Gap Fire,33867.0,G,NaN,NaN,1.0,Aug,NaN,...,18.03,54.17,0.0,2.0,1,parks_creek,Shasta,77.0,34.0,0.44
5,5,2016,Rey Fire,32606.0,G,NaN,NaN,28.0,Aug,NaN,...,6.08,10.28,26.0,3.0,0,,,0,0,0
6,6,2016,Cedar Fire,29322.0,G,NaN,NaN,46.0,Aug,NaN,...,2.15,4.88,52.0,0.0,1,mineral_king,Kaweah,36.0,16.0,0.44
7,7,2016,Canyon Fire,12518.0,G,NaN,NaN,10.0,Sep,NaN,...,3.84,8.76,0.0,0.0,0,,,0,0,0
8,8,2016,Pilot Fire,8110.0,G,NaN,NaN,9.0,Aug,NaN,...,3.41,6.45,43.0,0.0,0,,,0,0,0
9,9,2016,Border Fire,7609.0,G,NaN,NaN,11.0,Jun,NaN,...,7.63,9.31,1.0,5.0,0,,,0,0,0


In [303]:
wildfires1.to_csv('wildfires1_with_snow.csv', index = False)
wildfires2.to_csv('wildfires2_with_snow.csv', index = False)